In [ ]:
import os, sys
sys.path.append('../utils')
import pandas as pd
import numpy as np
from queryHelper import prodFetch, adbFetch
from databaseHelper import *
from sheetHelper import *
from datetime import timedelta
import psycopg2
from datetime import datetime
from openpyxl import Workbook

In [ ]:
dfDrivers = prodFetch("""select zoneId, count(distinct operatorId), count(id) from drivers where isBaaSDriver!=1 and isDeliveryDriver!=1 and vehicleType!='E-2w' and clientId = 'BS00' and zoneId = 'LKR08' and id like 'D%' and operatorId is not null and status in ('active', 'inactive')""")

dfDrivers

In [ ]:
dfDrivers3 = prodFetch("""select zoneId, count(distinct operatorId), count(id) from drivers where isBaaSDriver!=1 and isDeliveryDriver!=1 and vehicleType!='E-2w' and clientId = 'BS00' and zoneId = 'LKR08' and id like 'D%' and operatorId is null and status in ('active', 'inactive')""")

dfDrivers3

In [ ]:
dfDrivers2 = prodFetch("""select zoneId, count(id) from drivers where isBaaSDriver!=1 and isDeliveryDriver!=1 and vehicleType!='E-2w' and clientId = 'BS00' and zoneId = 'LKR08' and id like 'D%' and status in ('active', 'inactive')""")

dfDrivers2

In [ ]:
drivers = prodFetch("""select id from drivers where isBaaSDriver!=1 and isDeliveryDriver!=1 and vehicleType!='E-2w' and clientId = 'BS00' and zoneId = 'LKR08' and id like 'D%' and status in ('active', 'inactive')""")

driversList = tuple(drivers['id'].unique())

print(len(driversList))
driversList

In [ ]:
txns = prodFetch(f"""select driverId, date, count(*) txnCount from transactions where deletedAt is null and driverId in {driversList} and date >= date_add(current_date(), interval -30 day) group by driverId, date""")

txns

In [ ]:
grouped = txns.groupby(['driverId'], as_index=False).agg({
    'txnCount': 'mean'
})

grouped.rename(columns={'txnCount': 'dailyMeanTxns'}, inplace=True)

grouped

In [ ]:
grouped.nunique()

In [ ]:
oneSwap = grouped[grouped['dailyMeanTxns'] >= 2].nunique()
oneSwap

In [ ]:
oneSwap = grouped[grouped['dailyMeanTxns'] < 2].nunique()
oneSwap

In [ ]:
ticket = adbFetch(f"""select complainant_id, calling_number, mobile, alternate_mobile, primary_number, source from ticketModelNew where complainant_id in {driversList} and complainant_type = "driver" and date >= '20250308' and date < '20250408' and source in ('Inbound', 'Whatsapp', 'driverApp')""")

ticket

In [ ]:
# Fixing the row-wise comparison
ticket['Registered'] = ticket.apply(lambda row: 1 if row['calling_number'] == row['mobile'] else 0, axis=1)
ticket['Alternate Mobile'] = ticket.apply(lambda row: 1 if row['calling_number'] == row['alternate_mobile'] else 0, axis=1)
ticket['Primary'] = ticket.apply(lambda row: 1 if row['calling_number'] == row['primary_number'] else 0, axis=1)

# Others = not in any of the above categories
ticket['Others'] = (
    (ticket['Registered'] == 0) &
    (ticket['Alternate Mobile'] == 0) &
    (ticket['Primary'] == 0)
).astype(int)

ticket

In [ ]:
number_categories = {
    'Primary': 'Primary',
    'Others': 'Others',
    'Registered': 'Registered',
    'Alternate Mobile': 'Alternate Mobile'
}

summary_data = []

for col, label in number_categories.items():
    df_filtered = ticket[ticket[col] == 1]
    total_calls = len(df_filtered)
    unique_drivers = df_filtered['complainant_id'].nunique()
    percentage = round((total_calls / len(ticket)) * 100, 2)
    calls_per_driver = round(total_calls / unique_drivers, 2) if unique_drivers else 0

    summary_data.append({
        'number_category': label,
        'total_calls': total_calls,
        'unique_drivers': unique_drivers,
        'percentage': percentage,
        'calls per unique drivers': int(calls_per_driver)
    })

summary_df = pd.DataFrame(summary_data)

# Add total row
total_row = {
    'number_category': 'Total',
    'total_calls': summary_df['total_calls'].sum(),
    'unique_drivers': summary_df['unique_drivers'].sum(),
    'percentage': 100,
    'calls per unique drivers': round(summary_df['total_calls'].sum() / summary_df['unique_drivers'].sum())
}
summary_df = pd.concat([summary_df, pd.DataFrame([total_row])], ignore_index=True)

summary_df

In [ ]:
len(ticket)